In [ ]:
pip install pandas

In [ ]:
pip install librosa

In [ ]:
pip install soundfile

In [ ]:
pip install aifc

In [ ]:
import sys
print(sys.version)

In [ ]:
pip install audioread

In [ ]:
pip install pydub

In [ ]:
pip install ffmpeg

In [2]:
pip install torchaudio

Note: you may need to restart the kernel to use updated packages.Collecting torchaudio
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 2.4/2.4 MB 30.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/204.1 MB ? eta -:--:--
   -- ------------------------------------- 10.5/204.1 MB 50.5 MB/s eta 0:00:04
   ---- ----------------------------------- 24.9/204.1 MB 60.7 MB/s eta 0:00:03
   ------- -------------------------------- 37.7/204.1 MB 61.8 MB/s eta 0:00:03
   ---------- ----------------------------- 51.6/204.1 MB 63.2 MB/s eta 0:00:03
   ------------ --------------------------- 62.9/204.1 MB 61.7 MB/s eta 0:00:03
   -------------- ------------------------- 71.6/204.1 MB 58.5 MB/s eta 0:00:03
   --------------- ------------------------ 81.0/204.1 MB 56.8 MB/s eta 0:00:03
   ----------------- ---------------------- 89.7/204.1 MB 55.0 MB/s eta 0:00:03
   ------------------- -------------------- 99.4/204.1 M

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


CODE

In [ ]:
import os
import numpy as np
import pandas as pd
import librosa
import torchaudio

def extract_frequencies(mp3_file, hop_length_ms=10):
    """
    Extrait la fréquence dominante toutes les 10ms d'un fichier MP3
    """
    # Charger le fichier audio avec torchaudio
    waveform, sr = torchaudio.load(mp3_file)
    
    # Convertir en tableau numpy
    y = waveform.numpy().flatten()
    
    # Calcul du hop_length basé sur la fréquence d'échantillonnage
    hop_length = int(sr * hop_length_ms / 1000)
    
    # Calculer le spectrogramme de Fourier
    stft = np.abs(librosa.stft(y, hop_length=hop_length, n_fft=2048))
    
    # Extraire la fréquence dominante pour chaque trame
    freqs = librosa.fft_frequencies(sr=sr)
    dominant_freqs = freqs[np.argmax(stft, axis=0)]
    
    return dominant_freqs

def process_directory():
    """
    Parcourt tous les fichiers MP3 du dossier et génère les CSV avec des valeurs toutes les 10ms
    """
    for file in os.listdir('.'):
        if file.endswith('.mp3'):
            print(f"Traitement de {file}...")
            freqs = extract_frequencies(file)
            csv_filename = file.replace('.mp3', '.csv')
            df = pd.DataFrame(freqs)
            df.to_csv(csv_filename, index=False, header=False)
            print(f"Fichier sauvegardé : {csv_filename}")

if __name__ == "__main__":
    process_directory()

Traitement de ClairDeLune.mp3...
Fichier sauvegardé : ClairDeLune.csv


In [12]:
import os
import numpy as np
import pandas as pd
import torchaudio

def extract_frequencies(mp3_file, hop_length_ms=10):
    """
    Extrait la fréquence dominante toutes les 10ms d'un fichier MP3 en utilisant numpy.fft
    """
    # Charger le fichier audio avec torchaudio
    waveform, sr = torchaudio.load(mp3_file)
    
    # Convertir en tableau numpy
    y = waveform.numpy().flatten()
    
    # Calcul du hop_length basé sur la fréquence d'échantillonnage
    hop_length = int(sr * hop_length_ms / 1000)
    
    # Taille de la fenêtre pour la FFT (choisir une taille qui est une puissance de 2 pour une FFT plus rapide)
    window_size = 2048  # Taille de la fenêtre FFT
    
    # Nombre d'échantillons totaux
    total_samples = len(y)
    
    # Liste pour stocker les fréquences dominantes
    dominant_freqs = []
    
    # Parcourir les données audio par fenêtres glissantes
    for start in range(0, total_samples - window_size, hop_length):
        window = y[start:start + window_size]
        
        # Appliquer la FFT
        fft_result = np.fft.fft(window)
        
        # Calculer la fréquence associée à chaque indice de la FFT
        freqs = np.fft.fftfreq(window_size, d=1/sr)
        
        # Prendre seulement les fréquences positives
        positive_freqs = freqs[:window_size // 2]
        positive_fft = np.abs(fft_result[:window_size // 2])
        
        # Trouver l'indice de la fréquence dominante
        dominant_freq = positive_freqs[np.argmax(positive_fft)]
        dominant_freqs.append(dominant_freq)
    
    # Calcul du nombre de valeurs attendues
    hop_samples = int(sr * hop_length_ms / 1000)
    expected_values = total_samples // hop_samples  # Nombre attendu de trames (toutes les 10 ms)
    print(hop_samples)
    print(total_samples)

    # Affichage du nombre de valeurs extraites et attendu
    print(f"Nombre de valeurs extraites : {len(dominant_freqs)}, attendu : {expected_values}")
    
    return dominant_freqs

def process_directory():
    """
    Parcourt tous les fichiers MP3 du dossier et génère les CSV avec des valeurs toutes les 10ms
    """
    for file in os.listdir('.'):
        if file.endswith('.mp3'):
            print(f"Traitement de {file}...")
            freqs = extract_frequencies(file)
            csv_filename = file.replace('.mp3', '.csv')
            df = pd.DataFrame(freqs)
            df.to_csv(csv_filename, index=False, header=False)
            print(f"Fichier sauvegardé : {csv_filename}")

if __name__ == "__main__":
    process_directory()


Traitement de ClairDeLune.mp3...
441
8630784
Nombre de valeurs extraites : 19567, attendu : 19570
Fichier sauvegardé : ClairDeLune.csv


In [13]:
import os
import numpy as np
import pandas as pd
import torchaudio

def extract_frequencies(mp3_file, hop_length_ms=10):
    """
    Extrait la fréquence dominante toutes les 10ms d'un fichier MP3 en utilisant numpy.fft
    """
    # Charger le fichier audio avec torchaudio
    waveform, sr = torchaudio.load(mp3_file)
    
    # Convertir en tableau numpy
    y = waveform.numpy().flatten()
    
    # Calcul du hop_length basé sur la fréquence d'échantillonnage
    hop_length = int(sr * hop_length_ms / 1000)  # hop_length en échantillons
    
    # Taille de la fenêtre pour la FFT (choisir une taille qui est une puissance de 2 pour une FFT plus rapide)
    window_size = 2048  # Taille de la fenêtre FFT
    
    # Nombre d'échantillons totaux
    total_samples = len(y)
    
    # Liste pour stocker les fréquences dominantes
    dominant_freqs = []
    
    # Parcourir les données audio par fenêtres glissantes
    for start in range(0, total_samples - window_size, hop_length):
        window = y[start:start + window_size]
        
        # Appliquer la FFT
        fft_result = np.fft.fft(window)
        
        # Calculer la fréquence associée à chaque indice de la FFT
        freqs = np.fft.fftfreq(window_size, d=1/sr)
        
        # Prendre seulement les fréquences positives
        positive_freqs = freqs[:window_size // 2]
        positive_fft = np.abs(fft_result[:window_size // 2])
        
        # Trouver l'indice de la fréquence dominante
        dominant_freq = positive_freqs[np.argmax(positive_fft)]
        dominant_freqs.append(dominant_freq)
    
    # Calcul du nombre de valeurs attendues
    expected_values = total_samples // hop_length  # Nombre attendu de trames (toutes les 10 ms)
    
    # Affichage du nombre de valeurs extraites et attendu
    print(f"Nombre de valeurs extraites : {len(dominant_freqs)}, attendu : {expected_values}")
    
    return dominant_freqs

def process_directory():
    """
    Parcourt tous les fichiers MP3 du dossier et génère les CSV avec des valeurs toutes les 10ms
    """
    for file in os.listdir('.'):
        if file.endswith('.mp3'):
            print(f"Traitement de {file}...")
            freqs = extract_frequencies(file)
            csv_filename = file.replace('.mp3', '.csv')
            df = pd.DataFrame(freqs)
            df.to_csv(csv_filename, index=False, header=False)
            print(f"Fichier sauvegardé : {csv_filename}")

if __name__ == "__main__":
    process_directory()


Traitement de ClairDeLune.mp3...
Nombre de valeurs extraites : 19567, attendu : 19570
Fichier sauvegardé : ClairDeLune.csv


In [15]:
import os
import numpy as np
import pandas as pd
import torchaudio

def extract_frequencies(mp3_file, hop_length_ms=10):
    """
    Extrait la fréquence dominante toutes les 10ms d'un fichier MP3 en utilisant numpy.fft
    """
    # Charger le fichier audio avec torchaudio
    waveform, sr = torchaudio.load(mp3_file)
    
    # Convertir en tableau numpy
    y = waveform.numpy().flatten()
    
    # Calcul du hop_length basé sur la fréquence d'échantillonnage
    hop_length = int(sr * hop_length_ms / 1000)  # hop_length en échantillons
    
    # Taille de la fenêtre pour la FFT (choisir une taille qui est une puissance de 2 pour une FFT plus rapide)
    window_size = 2048  # Taille de la fenêtre FFT
    
    # Nombre d'échantillons totaux
    total_samples = len(y)
    
    # Liste pour stocker les fréquences dominantes
    dominant_freqs = []
    
    # Parcourir les données audio par fenêtres glissantes
    for start in range(0, total_samples - window_size, hop_length):
        window = y[start:start + window_size]
        
        # Appliquer la FFT
        fft_result = np.fft.fft(window)
        
        # Calculer la fréquence associée à chaque indice de la FFT
        freqs = np.fft.fftfreq(window_size, d=1/sr)
        
        # Prendre seulement les fréquences positives
        positive_freqs = freqs[:window_size // 2]
        positive_fft = np.abs(fft_result[:window_size // 2])
        
        # Trouver l'indice de la fréquence dominante
        dominant_freq = positive_freqs[np.argmax(positive_fft)]
        dominant_freqs.append(dominant_freq)
        
    return dominant_freqs

def process_directory():
    """
    Parcourt tous les fichiers MP3 du dossier et génère les CSV avec des valeurs toutes les 10ms
    """
    for file in os.listdir('.'):
        if file.endswith('.mp3'):
            print(f"Traitement de {file}...")
            freqs = extract_frequencies(file)
            csv_filename = file.replace('.mp3', '.csv')
            df = pd.DataFrame(freqs)
            df.to_csv(csv_filename, index=False, header=False)
            print(f"Fichier sauvegardé : {csv_filename}")

if __name__ == "__main__":
    process_directory()

Traitement de ClairDeLune.mp3...
Fichier sauvegardé : ClairDeLune.csv


In [16]:
import os
import numpy as np
import pandas as pd
import torchaudio

def extract_frequencies(mp3_file, hop_length_ms=10):
    """
    Extrait la fréquence dominante toutes les 10ms d'un fichier MP3 en utilisant numpy.fft
    """
    # Charger le fichier audio avec torchaudio
    waveform, sr = torchaudio.load(mp3_file)
    
    # Convertir en tableau numpy
    y = waveform.numpy().flatten()
    
    # Calcul du hop_length basé sur la fréquence d'échantillonnage
    hop_length = int(sr * hop_length_ms / 1000)  # hop_length en échantillons
    
    # Taille de la fenêtre pour la FFT (choisir une taille qui est une puissance de 2 pour une FFT plus rapide)
    window_size = 2048  # Taille de la fenêtre FFT
    
    # Nombre d'échantillons totaux
    total_samples = len(y)
    
    # Liste pour stocker les fréquences dominantes
    dominant_freqs = []
    
    # Parcourir les données audio par fenêtres glissantes
    for start in range(0, total_samples - window_size, hop_length):
        window = y[start:start + window_size]
        
        # Appliquer la FFT
        fft_result = np.fft.fft(window)
        
        # Calculer la fréquence associée à chaque indice de la FFT
        freqs = np.fft.fftfreq(window_size, d=1/sr)
        
        # Prendre seulement les fréquences positives
        positive_freqs = freqs[:window_size // 2]
        positive_fft = np.abs(fft_result[:window_size // 2])
        
        # Trouver l'indice de la fréquence dominante
        dominant_freq = positive_freqs[np.argmax(positive_fft)]
        dominant_freqs.append(dominant_freq)
    
    return dominant_freqs

def process_directory():
    """
    Parcourt tous les fichiers MP3 du dossier et génère les CSV avec des valeurs toutes les 10ms
    """
    for file in os.listdir('.'):
        if file.endswith('.mp3'):
            print(f"Traitement de {file}...")
            freqs = extract_frequencies(file)
            
            # Supprimer une valeur sur deux pour obtenir un taux de 10 ms
            reduced_freqs = freqs[::2]
            
            csv_filename = file.replace('.mp3', '.csv')
            df = pd.DataFrame(reduced_freqs)
            df.to_csv(csv_filename, index=False, header=False)
            print(f"Fichier sauvegardé : {csv_filename}")

if __name__ == "__main__":
    process_directory()

Traitement de ClairDeLune.mp3...
Fichier sauvegardé : ClairDeLune.csv
